## Рекомендация тарифов

В моем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
import pandas as pd # чтение файла
from sklearn.model_selection import train_test_split  
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier   
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
def data_describe(dataframe):
    display(dataframe.shape)
    display(dataframe.head(10))

In [4]:
data_describe(data)

(3214, 5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


**У нас данные из прошлого проекта, все данные обработаны
<br> 5 столбцов, 3214 строк.
<br> В столбце `is_ultra` значение 1 - пользуется тарифом "Ультра", 0 - пользуется тарифом "Смарт"**



## Разбиение данных на выборки

In [5]:
features = data.drop('is_ultra', axis=1)#  признаки из данных
target = data['is_ultra'] # целевой признак

In [6]:
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.25, 
                                                                              stratify = target,
                                                                              random_state=12345)
#выделяю 25% данных для валидации

In [7]:
features_train, features_test, target_train, target_test = train_test_split(features_train, 
                                                                            target_train, 
                                                                            test_size=0.25, 
                                                                            stratify = target_train,
                                                                            random_state=12345)
#выделяю 25% данных для тестовой выборки, из обучающей

In [8]:
display(features.shape)
display(features_train.shape)
display(features_valid.shape)
display(features_test.shape)

(3214, 4)

(1807, 4)

(804, 4)

(603, 4)

## Исследуйте модели

**Ищю лучшую модель среди деревьев**

In [9]:
best_model_tree = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions) 
    if result > best_result:
        best_model_tree = model
        best_result = result
        best_depth = depth
        
print("Accuracy лучшей модели:", best_result)
print("Количество веток:", best_depth)


Accuracy лучшей модели: 0.8009950248756219
Количество веток: 6


**Хороший результат, и модель дерева из 5 ответвлений можно считать не `не доученной` или `переобученной`**

**Следующий цикл ищет лучшую модель леса деревьев (код выполняется почти минуту, но показывает результат близкий к двум другим методам)**

In [10]:
best_model_forest = None
best_result = 0
best_est = 0
best_depth = 0

for est in range(1, 51):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model_forest = model
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Количество деревьев:", best_est,)
print("Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.8171641791044776
Количество деревьев: 4
Максимальная глубина: 10


**Случайный лес показал результат лучше - `0.81` (лутший вариант из 4 деревьев глубиной 10 веток)**

**Проверка модели логической регрессии с максимальным числом итераций до 1000**(код выполняется около 30 секунд)

In [11]:
best_model_logreg = None
best_result = 0
best_iter = 0

for i in range(100, 1001, 50):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=i) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid) 
    if result > best_result:
            best_model_logreg = model
            best_result = result
            best_iter = i
            

print("Accuracy логической регрессии:", best_result )
print("Максимальное кол-во итераций:", best_iter)

Accuracy логической регрессии: 0.746268656716418
Максимальное кол-во итераций: 100


При разном количествое итераций логическая регрессия даёт лучший результат на оптимальном количестве итераций в 100 единиц)

**Исследуя 3 вида моделей на обучающей выборке можно сделать вывод что:
<br>Лес деревьев выдает свой лучший результат `ACC - 0.81` с лесом деревьев из 4 деревьев и 10 веток
<br>лучший показатель точности у одиночного дерева из 5 ветвей `ACC - 0.8` - глубина 6
<br>У модели логической регресии `0.74`, количество итераций может быть стандартное**

<br> **У нас лучше всего показала себя модель `Случайного леса`, модель из 4 деревьев с глубиной в 10 веток, в исследовании буду использовать данную модель**


## Проверьте модель на тестовой выборке

In [15]:
predictions = best_model_forest.predict(features_test) 
result = accuracy_score(target_test, predictions) 
print('Точность модели леса из 4 деревьев глубиной 10 : ',result)


Точность модели леса из 4 деревьев глубиной 10 :  0.7810945273631841


**Вывод:**

На тестовой выборке показатели качество модели Случайного леса чуть меньше 0.78, чем при обучении - 0.81, но результат тоже хороший и модель можно считать рабочей


## Проверка модели на адекватность

In [13]:
model = DummyClassifier(strategy="most_frequent")
model.fit(features_test, target_test)
model.predict(features_test)
model.score(features_test, target_test)

0.693200663349917

**Вывод:**
При проверке на адекватность моделей - модель "дерева" показаль качество в 0,69, средний показатель,так что можно утверждать на адекватности нашей модели

## Вывод

<br>Проведя исследования между 3 моделями мне удалось выделить модель которая показывать точность в 0.81 - это `модель Случайного леса` из 4 деревьев и глубиной в 10 ответвлений на обучении, на тестовой выборке качество немного упало до 0.78, но в рамках допустимого
<br>Так же я увидел что самая быстрая `модель Логической регрессии` показала средний результат с точностью в 0.7
<br>модель `Дерева` показала качество обучения 0.8
<br> Адекватностивыбранной модели `Случайного леса` считаю оправданной, так как качество модели при методе всегда возвращающем наиболее часто встречающуюся метку класса - 0.69
<br>